In [2]:
import os
import cv2
import numpy as np
from PIL import Image, ImageChops

def compute_image_difference(img1, img2):
    """Compute the root mean square difference between two images."""
    # print("Computing image difference...")
    diff = ImageChops.difference(img1, img2)
    h = diff.histogram()
    sq = (value * ((idx % 256) ** 2) for idx, value in enumerate(h))
    sum_of_squares = sum(sq)
    rms = (sum_of_squares / float(img1.size[0] * img1.size[1])) ** 0.5
    # print(f"Image difference was {rms:.3f}")
    return rms

def find_similar_images(directory):
    """Find the most similar images in the directory."""
    print("Finding similar images...")
    images = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.png')]
    similar_pairs = []
    
    print(f"Checking {len(images)} images...")
    for i in range(len(images)):
        for j in range(i + 1, len(images)):
            img1 = Image.open(images[i])
            img2 = Image.open(images[j])
            diff = compute_image_difference(img1, img2)
            similar_pairs.append((images[i], images[j], diff))
    
    similar_pairs.sort(key=lambda x: x[2])
    return similar_pairs

def resize_images_to_same_size(images, size):
    """Resize images to the same size."""
    print(f"Resizing all {len(images)} images")
    for img_path in images:
        img = Image.open(img_path)
        img = img.resize(size, Image.ANTIALIAS)
        img.save(img_path)

def create_video_from_images(images, output_video_path, fps=1):
    """Create a video from a list of images."""
    print("Creating video from images...")
    frame = cv2.imread(images[0])
    height, width, layers = frame.shape
    video = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    print("Writing frames...")
    for image in images:
        video.write(cv2.imread(image))

    print("Writing video complete.")
    video.release()

def main():
    image_directory = 'C:\\Users\\deste\\Pictures\\Saved Pictures\\pics\\'
    print(f"Warning this may resize any files in {image_directory}")
    output_video_path = 'output_video.mp4'
    fps = 2  # Frames per second

    # Step 1: Find similar images
    similar_images = find_similar_images(image_directory)
    most_similar_images = [pair[0] for pair in similar_images[:10]]  # Select top 10 similar images

    # Step 2: Resize images to the same size
    first_image = Image.open(most_similar_images[0])
    target_size = first_image.size
    resize_images_to_same_size(most_similar_images, target_size)

    # Step 3: Create video from images
    create_video_from_images(most_similar_images, output_video_path, fps)

if __name__ == "__main__":
    main()

Warning this may resize any files in C:\Users\deste\Pictures\Saved Pictures\pics\
Finding similar images...
Checking 238 images...
Resizing all 10 images


AttributeError: module 'PIL.Image' has no attribute 'ANTIALIAS'

In [1]:
import torch
import numpy as np
import os
from PIL import Image
from tqdm.auto import tqdm
from diffusers import StableDiffusionPipeline, DDIMScheduler
import cv2

def load_images(image_paths):
    return [Image.open(img_path) for img_path in image_paths]

def generate_intermediate_frames(img1, img2, model, num_frames=10):
    """Generate intermediate frames using a diffusion model."""
    frames = []
    for i in range(num_frames):
        alpha = i / float(num_frames)
        interpolated_latent = (1 - alpha) * img1 + alpha * img2
        frame = model.decode(interpolated_latent)
        frames.append(frame)
    return frames

def create_video_from_frames(frames, output_video_path, fps=20):
    """Create a video from a list of frames."""
    height, width, _ = frames[0].shape
    video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for frame in frames:
        video_writer.write(cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR))
    video_writer.release()

def main():
    # Paths to input images
    image_directory = 'C:\\Users\\deste\\Pictures\\Saved Pictures\\pics\\'
    print(f"Warning this may resize any files in {image_directory}")
    output_video_path = 'output_video.mp4'
    fps = 20

    # Load the diffusion model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to(device)
    model.scheduler = DDIMScheduler.from_config(model.scheduler.config)

    # Load images and find similar pairs
    image_files = [os.path.join(image_directory, f) for f in os.listdir(image_directory) if f.endswith('.png')]
    images = load_images(image_files)
    
    all_frames = []
    for i in range(len(images) - 1):
        img1 = model.encode_image(images[i].convert("RGB"), device, 1).latent_dist.sample()
        img2 = model.encode_image(images[i + 1].convert("RGB"), device, 1).latent_dist.sample()
        intermediate_frames = generate_intermediate_frames(img1, img2, model, num_frames=10)
        all_frames.extend(intermediate_frames)
    all_frames.append(images[-1])

    # Create and save the video
    create_video_from_frames(all_frames, output_video_path, fps)

if __name__ == '__main__':
    main()

Warning this may resize any files in C:\Users\deste\Pictures\Saved Pictures\pics\


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

AttributeError: 'NoneType' object has no attribute 'parameters'